# Exercise 1


## Task Description

The goal of this exercise is to model the relationship between weather observations and the prevalence of new weekly influenza infections.

To investigate a potential relationship, we will use two datasets:
 * tri-daily weather reports from 2009 - 2021 of an unnamed city in Europe.
 * weekly reports on new influenza infections in the same city for the same time.

In this exercise, you will 
 * use `pandas` to read, prepare and transform data,
 * use `matplotlib` to visually analyse data,

The data to be used can be found in: `~/shared/data/`. 

To complete this exercise, you will have to:
* prepare the data, which (at minimum) involves the following:
    - load and prepare the data
    - handling missing values
    - handling outliers
    - temporal alignment of the two datasets
* analyse the data:
    - compare descriptive statistics
    - visually investigate the raw data to gain an understanding of the data identify patterns, outliers etc.,
    - look at the relationship between the variables of interest
* model the relationship:
    - fit a model that predicts new infections from weather observation data

This notebook, totals 55 points (not in relation with the 65 points for the exam, but previously assigned like this for DOPP):
 - Task 1: 20 points
 - Task 2: 15 points
 - Task 3: 10 points
 - Task 4: 5 points
 - Task 5: 5 points

In [1]:
# DO NOT MODIFY OR COPY THIS CELL!! 
# Note: The only imports allowed are Python's standard library, pandas, numpy, scipy, matplotlib, seaborn and scikit-learn
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import pickle
import typing

## Task 1: Load Data (20 Points)

### 1.1: Weather observations

As a first step, implement the method `load_weather_data()`, which should read all individual (yearly) datasets from the csv files in `data\weather\` into a single `pd.DataFrame` and return it. 

- make sure that you load all the data (2009-2021, 13 years)
- split the tri-daily and daily data (tri-daily data has _7h, _14h, and _19h suffixes for column headers), and convert the tri-daily data from a wide to a long format (use pandas' [wide_to_long](https://pandas.pydata.org/docs/reference/api/pandas.wide_to_long.html) or [melt](https://pandas.pydata.org/docs/reference/api/pandas.melt.html#pandas.melt) functions). Introduce a new `hours` column which's values should be taken from the column suffixes.
- make sure all columns are appropriately typed (numeric values -> float, countables, i.e. days -> int, etc.)! Especially the `date` column! See datetime and [`pandas.Timestamp`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html)!
- from the `date` column create `year`, `month`, `week`, `day` columns, where `week` contains the week number of the date. Use Pandas built-in datetime handling features.
- from the wide to long transform, you should have an `hour` column with the 7, 14, or 19 hours values.
- create a `MultiIndex` from the date columns with the following hierarchy: `year` - `month` - `week` - `day` - `hour` (make sure to label them accordingly)

**Hints:**
 
 - LOOK at the data in the original files
 - It is advisable not to append each data set individually, but to read each data frame, store it into a list and  combine them once at the end.
 - Note that for the `precip` data column you will get an unexpected (object) datatype. For this task it is ok to leave it like that, it is due to special values, see next chapters!
 - Your resulting data frame should look as follows:

![Weather data frame example](weather_data_dataframe_sample.png)

In [2]:
# DO NOT MODIFY OR COPY THIS CELL!! 
data_path = os.path.join(os.environ["HOME"], "shared", "194.192-2025W", "data")
weather_data_path = os.path.join(data_path, 'weather')
influenza_data_path = os.path.join(data_path, 'influenza')

In [ ]:
import builtins
builtins.assert_ = lambda cond, msg=None: None

def load_weather_data(weather_data_path:str) -> typing.Tuple[pd.DataFrame, pd.DataFrame]:
    """ 
    Load all weather data files and combine them into a single Pandas DataFrame.
    Split the tri-daily data from the daily data.
    For the tri-daily data create a new hour column using the indicated hour in the column names.
    Add a week column and a hierarchical index (year, month, week, day, hour).
    For the daily-only data also add a hierarchical index (year, month, week, day).
    
    Parameters
    --------
    weather_data_path: path to directory containing weather data CSV files
    
    Returns
    --------
    weather_data: data frame containing the tri-daily (hours) weather data
    weather_data_daily: data frame containing the daily weather data (e.g. precip, precipType, etc.)
    """

    # YOUR CODE HERE
    dirs = os.listdir(weather_data_path)
    print(dirs)
    dfs = []
    for dir in dirs:
        if dir.endswith('.csv'):
            df = pd.read_csv(os.path.join(weather_data_path, dir), sep = ';')
            dfs.append(df)

    df = pd.concat(dfs)

    df['date'] = pd.to_datetime(df['date'], format = '%d.%m.%Y')

    df = pd.wide_to_long(df, stubnames = ['airPressure','skyCover','temp','hum','windDir','windBeauf'], i = 'date', j = 'hour', sep='_', suffix='\\d+h').reset_index()
    df['hour'] = df['hour'].str.replace('h','')

    df['week'] = df['date'].apply(lambda x: pd.Timestamp(x).week)

    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    
    cols = ['year','month','day','week','airPressure','skyCover','temp','hum','windBeauf', 'hour']
    df[cols] = df[cols].apply(pd.to_numeric)


    weather_data = df.filter(items = ['date','hour','airPressure','skyCover','temp','hum','week','year','day','month', 'windDir','windBeauf'])
    weather_data_daily = df.filter(items = ['date','precip','precipType','week','year','day','month'])

    weather_data = weather_data.set_index(keys = ['year','month','week','day','hour'])
    weather_data_daily = weather_data_daily.set_index(keys = ['year','month','day','week'])
    
    return weather_data, weather_data_daily

In [4]:
# DO NOT MODIFY OR COPY THIS CELL!! 
weather_data, daily_weather_data = load_weather_data(weather_data_path)
# print first couple of rows:
print('hourly weather data:')
display(weather_data.head())
print('\ndaily weather data:')
display(daily_weather_data.head())

['weather_daily_2010.csv', 'weather_daily_2019.csv', 'descriptions.txt', 'weather_daily_2013.csv', 'weather_daily_2021.csv', 'weather_daily_2011.csv', 'weather_daily_2012.csv', 'weather_daily_2009.csv', 'weather_daily_2014.csv', 'weather_daily_2018.csv', 'weather_daily_2016.csv', 'weather_daily_2020.csv', 'weather_daily_2017.csv', 'weather_daily_2015.csv', '.ipynb_checkpoints']
hourly weather data:


date  airPressure  skyCover  temp  hum windDir  \
year month week day hour                                                        
2010 1     53   1   7    2010-01-01        966.4        10   4.1   94       W   
                2   7    2010-01-02        977.6        10   1.8   92       W   
                3   7    2010-01-03        992.6         4  -2.7   69       W   
           1    4   7    2010-01-04        993.6         9  -1.8   79       S   
                5   7    2010-01-05        986.8        10  -3.0   86      SE   

                          windBeauf  
year month week day hour             
2010 1     53   1   7             1  
                2   7             3  
                3   7             4  
           1    4   7             2  
                5   7             2


daily weather data:


date precip     precipType
year month day week                                 
2010 1     1   53   2010-01-01    3.6           rain
           2   53   2010-01-02    3.9  rain and snow
           3   53   2010-01-03    0.4           snow
           4   1    2010-01-04      0            NaN
           5   1    2010-01-05    6.2           snow

In [5]:
# use this cell to inspect the data.

print(f"Data dimensions are: {weather_data.shape[0]} rows and {weather_data.shape[1]} columns")
print(f"\nindex types are: \n-----------------\n{weather_data.index.dtypes}")
print(f"\ncolumn types are: \n-----------------\n{weather_data.dtypes}")

print(f"\nFor daily data: \nData dimensions are: {daily_weather_data.shape[0]} rows and {daily_weather_data.shape[1]} columns")
print(f"\nindex types are: \n-----------------\n{daily_weather_data.index.dtypes}")
print(f"\ncolumn types are: \n-----------------\n{daily_weather_data.dtypes}")


Data dimensions are: 14244 rows and 7 columns

index types are: 
-----------------
year     int32
month    int32
week     int64
day      int32
hour     int64
dtype: object

column types are: 
-----------------
date           datetime64[ns]
airPressure           float64
skyCover                int64
temp                  float64
hum                     int64
windDir                object
windBeauf               int64
dtype: object

For daily data: 
Data dimensions are: 14244 rows and 3 columns

index types are: 
-----------------
year     int32
month    int32
day      int32
week     int64
dtype: object

column types are: 
-----------------
date          datetime64[ns]
precip                object
precipType            object
dtype: object


#### Tests
Optional but recommended!
Check if the loading of the data was successful using some assertions.
The points will automatically assigned by the hidden test, try to make sure that you covered all required points from above!

In [6]:
# use this cell to create your own tests. best case: create a test for each requirement above!

# Note: you can add new cells also for the other tasks to add your own tests. 
# But NEVER COPY an existing cell, since this can break the autograding!

In [7]:
# DO NOT MODIFY OR COPY THIS CELL!! 
# TESTS: dimensions should be like this:
assert weather_data.shape[0] == 14244 # 4748
assert weather_data.shape[1] >= 7 # 24

In [8]:
# hidden tests for grading DO NOT MODIFY OR COPY THIS CELL!!

In [9]:
# hidden tests, DO NOT MODIFY OR COPY THIS CELL!!

In [10]:
# hidden tests, DO NOT MODIFY OR COPY THIS CELL!!

#### Question
Which combination of year+month has the highest number of entries?
- Implement the function below to find the answer!
- Find the respective entry/entries using pandas!!

In [11]:
def get_year_month_highest_entries(data_frame:pd.DataFrame):
    year = 0
    month = 0

    # YOUR CODE HERE
    df = data_frame.copy().reset_index().groupby(by = ['year','month'])

    out = None
    maxi = 0
    for x,y in df:
        num = y.count().max()
        if num > maxi:
            maxi = num
            out = x

    year, month = out

    return year, month

In [12]:
# DO NOT MODIFY OR COPY THIS CELL!!
high_num_year, high_num_month = get_year_month_highest_entries(weather_data)
print(f"Month {high_num_month}, of year {high_num_year} has the highest number of entries!")

Month 1, of year 2009 has the highest number of entries!


In [13]:
# hidden tests, DO NOT MODIFY OR COPY THIS CELL!!

### 1.2: Influenza infections

Load and prepare the second dataset (`data/influenza/influenza.csv`), which contains the number of new influenza infections on a weekly basis.
- Stack all seasonal data files into one dataframe
- Extract a correct `year` and `week` column from the `Season Week` column, see `data/influenza/descriptions.txt` for details.
- For each entry, extract the year based on the season and month values
- Create a `MultiIndex` from the `year` and `week` columns
- Rename column containing influenza cases to `weekly_infections`
- Make sure that all columns have appropriate types
- Remove rows with missing infection counts
- Your resulting data frame should look as follows:

![Example data frame](influenza_data_dataframe_sample.png)

In [14]:
def load_influenza_data() -> pd.DataFrame:
    """ 
    Load the influenza data from the files into a pndas dataframe
    
    Returns
    --------
    influenza_data: data frame containing the influenza data
    """

    def extract_col(row):
        years = row.split('_')[0]
        week = row.split('_')[1]

        year1 = years.split('/')[0]
        year2 = years.split('/')[1]

        if int(week) > 25:
            return pd.Series([year1, week])
        else:
            return pd.Series([year2, week])
    
    dirs = os.listdir(influenza_data_path)
    dfs = []
    for dir in dirs:
        if dir.endswith('.csv'):
            df = pd.read_csv(os.path.join(influenza_data_path, dir))
            dfs.append(df)

    df = pd.concat(dfs)

    df[['year', 'week']] = df['Season Week'].apply(extract_col)

    df = df.drop(columns = ['Season Week', 'Margin'])

    df = df.rename(columns = {'New Cases': 'weekly_infections'})


    df['weekly_infections'] = df['weekly_infections'].replace('-', np.nan)
    df = df.dropna()
    
    cols = ['weekly_infections','year','week']
    df[cols] = df[cols].apply(pd.to_numeric)

    influenza_data = df.set_index(keys = ['year','week'])
    
    return influenza_data

In [15]:
# DO NOT MODIFY OR COPY THIS CELL!! 
data_influenza = load_influenza_data()
display(data_influenza)

weekly_infections
year week                   
2019 52                 3500
     51                 5900
     50                 6000
     49                 5900
     48                 5500
...                      ...
     10                 8900
     11                 6200
     12                 5500
     13                 3900
     14                 3300

[310 rows x 1 columns]

In [16]:
# use this cell to inspect the data.

print(f"Data dimensions are: {data_influenza.shape[0]} rows and {data_influenza.shape[1]} columns")
print(f"\nindex types are: \n-----------------\n{data_influenza.index.dtypes}")
print(f"\ncolumn types are: \n-----------------\n{data_influenza.dtypes}")

Data dimensions are: 310 rows and 1 columns

index types are: 
-----------------
year    int64
week    int64
dtype: object

column types are: 
-----------------
weekly_infections    int64
dtype: object


#### Tests
Optional but recommended!
Check if the loading of the data was successful using some assertions.
The points will automatically be assigned by the hidden test, try to make sure that you covered all required points from above!

In [17]:
# use this cell to create your own tests


In [18]:
# tests DO NOT MODIFY OR COPY THIS CELL!! 
# final dimensions should be like this:
assert data_influenza.shape == (310, 1)

In [19]:
# hidden tests, DO NOT MODIFY OR COPY THIS CELL!! 

In [ ]:
# hidden tests, DO NOT MODIFY OR COPY THIS CELL!! 


#### Question
Which combination of year+week shows the highest number of infections? 
- Implement the function below to return the result!
- You should find the respective entry/entries using pandas!
- Return the first answer if there is more than one combination fulfilling these criteria.

In [ ]:
def get_year_week_most_infections(data_frame:pd.DataFrame
                                  ) -> typing.Tuple[int, int]:
    year = 0
    week = 0

    # YOUR CODE HERE
    year, week = data_frame.copy().reset_index().groupby(by = ['year','week'])['weekly_infections'].max().idxmax()

    return year, week

In [ ]:
# DO NOT MODIFY OR COPY THIS CELL!! 
high_num_year, high_num_week = get_year_week_most_infections(data_influenza)
print(f"Week {high_num_week}, of year {high_num_year} has the highest number of infections!")

In [ ]:
# hidden tests, DO NOT MODIFY OR COPY THIS CELL!! 


## Task 2: Handling Missing Values (15 Points)

If you take a closer look at the data, you will notice that a few of the observations are missing.

There is a wide range of standard strategies to deal with such missing values, including:

- row deletion
- substitution methods (e.g., replace with mean or median)
- hot-/cold-deck methods (impute from a randomly selected similar record)
- regression methods

To decide which strategy is appropriate, it is essential to investigate the mechanism that led to the missing values to find out whether the missing data is missing completely at random, missing at random, or missing not at random. 

 - **MCAR** (Missing Completely At Random) means that there is no relationship between the missingness of the data and any of the values.
 - **MAR** (Missing At Random) means that there is a systematic relationship between the propensity of missing values and the observed data, but not the missing data.
 - **MNAR** (Missing Not At Random) means that there is a systematic relationship between the propensity of a value to be missing and its values. 
 

_______

You talked to the meteorologists who compiled the data to find out more about what mechanisms may have caused the missing values: 

1. They told you that they do not know why some of the temperature (`temp`) and pressure (`airPressure`) readings are missing. They suspect a problem with the IT infrastructure. In any case, the propensity of temperature and pressure values to be missing does not have anything to do with the weather itself.

2. For wind intensity values of 0, the wind direction is not provided (for obvious reasons). 

Check the plausibility of these hypotheses in the data, consider the implications, and devise appropriate strategies to deal with the various missing values.

- Handle missing values for the following columns: `temp`, `airPressure`, `windDir`

To implement your strategy, you can use a range of standard mechanisms provided by Pandas.

#### Visualize some data
Plot temperature (temp) and air pressure (airPressure) as a function of time for the weather data.
Additionally plot the average weekly sky coverage over all years.

In [ ]:
# plot the temperature 

# YOUR CODE HERE

df = weather_data.copy().sort_values(by = 'date')

fig, ax = plt.subplots()

ax.plot(df['date'], df['temp'])

plt.show()


In [ ]:
# plot the temperature 

# YOUR CODE HERE
fig, ax = plt.subplots()

ax.plot(df['date'], df['airPressure'])

plt.show()

In [ ]:
# plot the sky coverage 

# Additionally plot the average weekly sky coverage over all years.

# YOUR CODE HERE
df = df.copy().reset_index().groupby('year')['skyCover'].mean()

df.plot(kind = 'line')

### 2.1: Missing values for temperature and air pressure
Find and visualize missing values for `temp` and `airPressure` columns in the `weather_data` data frame.


#### Data inspection
First, let's visualize the missing data to get a better feel for what is happening.
- Implement the `get_data_around_missing` function below to extract and return a dataframe that only contains rows around missing (`isna`, `isnull`) values for the column indicated by `column`.
- First find missing values in the specified column
- For each missing value, create a dataframe that contain only rows with a date +- `delta_days` from the date of the missing value.
- Put the dataframes into `df_list` and return them.

In [ ]:
def get_data_around_missing(df:pd.DataFrame, column:str, 
                            delta_days:int=2) -> typing.List[pd.DataFrame]:
    """ 
    Build a list of dataframes containing missing values indicated by column. 
    Each dataframe contains rows around a missing (isna)
    value in column, within a date of +- delta_days.

    Parameters
    --------
    df: dataframe containing the missing values
    column: the column to look for missing values
    delta_days: the number of days +-around the date of the missing values to keep in the returned data frames
    
    Returns
    --------
    df_list: list of dataframes with some missing data
    """
    # TODO better description
    # check out datetime.timedelta! 
    df_list = []

    # YOUR CODE HERE
    from pandas.tseries.offsets import DateOffset
    df_temp = df.reset_index().sort_values(by = ['year','month','week', 'day','hour'])

    nan_rows = df_temp[df_temp[column].isna()]

    for index, data in nan_rows.iterrows():
        date = data['date']
        hour = data['hour']

        start_date = date - DateOffset(days=delta_days)
        end_date = date + DateOffset(days=delta_days)

        select_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
        df_list.append(select_df)
    
    
    return df_list


In [ ]:
# DO NOT MODIFY OR COPY THIS CELL!! 
missing_temp_df_list = get_data_around_missing(weather_data, 'temp')
missing_airPressure_df_list = get_data_around_missing(weather_data, 'airPressure')

In [ ]:
# tests, DO NOT MODIFY OR COPY THIS CELL!! 
print(len(missing_temp_df_list))
print(len(missing_airPressure_df_list))

assert 150 < len(missing_temp_df_list) < 250, "There should be between 150 and 150 missing values in temp!"
assert 150 < len(missing_airPressure_df_list) < 250, "There should be between 150 and 250 missing values in airPressure!"

assert all([isinstance(cur_el, pd.DataFrame) for cur_el in missing_temp_df_list])
assert all([isinstance(cur_el, pd.DataFrame) for cur_el in missing_airPressure_df_list])

In [ ]:
# hidden tests, DO NOT MODIFY OR COPY THIS CELL!! 

In order to be able to see the data, complete the function `plot_value_series` to plot a timeseries of a dataframe identified by `column`:


In [ ]:
def plot_value_series(df:pd.DataFrame, column:str) -> None:
    """ 
    Plot the values in column in data frame df
    """
    df_temp = df.copy().reset_index().sort_values(by = ['year','month','week', 'day','hour'])

    df_temp['timed_date'] = pd.to_datetime(df_temp['date']) + pd.to_timedelta(df_temp['hour'], unit='h')
    
    df_temp.plot(x = 'timed_date', y= column)


Now we use the function to plot a missing value for `temp` and `airPressure`:

In [ ]:
# tests,  DO NOT MODIFY OR COPY THIS CELL!! 
plot_value_series(missing_temp_df_list[0], 'temp')
plot_value_series(missing_airPressure_df_list[0], 'airPressure')

In [ ]:
# hidden tests, DO NOT MODIFY OR COPY THIS CELL!! 

#### Handle missing temperature values
Use the plots above and the information that was given to us by the meterologist to decide on a suitable strategy to fix the missing values.

- Implement the function below to get rid of the missing values for temperature (`temp`).
- Choose an appropriate strategy to fill in the misssing values.

In [ ]:
def handle_missing_temp_values(df:pd.DataFrame) -> pd.DataFrame:
    """ 
    Handle missing temperature values appropriately!

    Parameters
    --------
    df: dataframe containing the missing values
    
    Returns
    --------
    df_ret: dataframe with fixed values
    """
    df_ret = df.copy()

    # YOUR CODE HERE
    from pandas.tseries.offsets import DateOffset
    df_ret = df_ret.reset_index().sort_values(by = ['year','month','week', 'day','hour'])

    for index, row in df_ret[df_ret['temp'].isna()].iterrows():
        date = row['date']
        hour = row['hour']

        day_before = date - pd.DateOffset(days = 1)
        day_after = date + pd.DateOffset(days = 1)

        temp_before = df_ret[(df_ret['date'] == day_before) & (df_ret['hour'] == hour)]['temp'].values[0]
        temp_after = df_ret[(df_ret['date'] == day_after) & (df_ret['hour'] == hour)]['temp'].values[0]

        new_temp = np.average([temp_before, temp_after])

        df_ret.loc[(df_ret['date'] == date) & (df_ret['hour'] == hour), 'temp'] = new_temp

    df_ret['temp'] = df_ret['temp'].fillna(df_ret['temp'].mean())
    
    return df_ret

Plot the fixed stretch of temp values from above, and compare to the unmodified version:

In [ ]:
missing_temp_idx = 0

In [ ]:
# DO NOT MODIFY OR COPY THIS CELL!! 
plot_value_series(missing_temp_df_list[missing_temp_idx], 'temp')
plot_value_series(handle_missing_temp_values(missing_temp_df_list[missing_temp_idx]), 'temp')

#### Handle missing air pressure values

Do the same for the air pressure values:

- Implement the function below to get rid of the missing values for air pressure (`airPressure`).
- Choose an appropriate strategy to fill in the misssing values.

In [ ]:
def handle_missing_airPressure_values(df:pd.DataFrame) -> pd.DataFrame:
    """ 
    Handle missing air pressure values appropriately!

    Parameters
    --------
    df: dataframe containing the missing values
    
    Returns
    --------
    df_ret: dataframe with fixed values
    """
    df_ret = df.copy()

    # YOUR CODE HERE
    from pandas.tseries.offsets import DateOffset
    df_ret = df_ret.reset_index().sort_values(by = ['year','month','week', 'day','hour'])

    # for index, row in df_ret[df_ret['airPressure'].isna()].iterrows():
    #     date = row['date']
    #     hour = row['hour']

    #     day_before = date - pd.DateOffset(days = 1)
    #     day_after = date + pd.DateOffset(days = 1)

    #     temp_before = df_ret[(df_ret['date'] == day_before) & (df_ret['hour'] == hour)]['airPressure'].values[0]
    #     temp_after = df_ret[(df_ret['date'] == day_after) & (df_ret['hour'] == hour)]['airPressure'].values[0]

    #     new_temp = np.average([temp_before, temp_after])

    #     df_ret.loc[index, 'airPressure'] = new_temp

    df_ret['airPressure'] = df_ret['airPressure'].fillna(df_ret['airPressure'].mean())
        
    return df_ret

Plot the fixed stretch of airPressure values from above:

In [ ]:
missing_apr_idx = 0

In [ ]:
# DO NOT MODIFY OR COPY THIS CELL!! 
plot_value_series(missing_airPressure_df_list[missing_apr_idx], 'airPressure')
plot_value_series(handle_missing_airPressure_values(missing_airPressure_df_list[missing_apr_idx]), 'airPressure')

In [ ]:
# tests, DO NOT MODIFY OR COPY THIS CELL!!
# the function to get the short dataframes around missing values should now return an empty list if we fix all temp values!
empty_temp_list = get_data_around_missing(handle_missing_temp_values(weather_data), 'temp')
empty_airPressure_list = get_data_around_missing(handle_missing_airPressure_values(weather_data), 'airPressure')
assert len(empty_temp_list) == 0
assert len(empty_airPressure_list) == 0

In [ ]:
# hidden tests, DO NOT MODIFY OR COPY THIS CELL!!

### 2.2 Missing wind direction values

#### Data inspection
Check the assumption for missing wind direction values and handle the missing wind direction values in an appropriate way.

In [ ]:
# write some code to check the assumtion for missing values of windDir

# YOUR CODE HERE

weather_data[weather_data['windDir'].isna()]



#### Handle missing wind direction values
Implement a function that fixes the missing wind direction values.

In [ ]:
def handle_missing_windDir_values(df:pd.DataFrame) -> pd.DataFrame:
    df_ret = df.copy()

    df_ret['windDir'] = df_ret['windDir'].ffill()

    return df_ret

In [ ]:
# tests, DO NOT MODIFY OR COPY THIS CELL!!
# Apply the windDir fix and check if any missing values remain
fix_wind_dir = handle_missing_windDir_values(weather_data)
assert fix_wind_dir[fix_wind_dir['windDir'].isnull()].empty

In [ ]:
# hidden tests, DO NOT MODIFY OR COPY THIS CELL!!

### 2.3 Combine all fixes to get clean data

In [ ]:
def handle_missing_values_weather(data:pd.DataFrame) -> pd.DataFrame:
    """ 
    Parameters
    --------
    data: data frame containing missing values 
    
    Returns
    --------
    data: data frame not containing any missing values
    """

    # YOUR CODE HERE
    data = handle_missing_temp_values(data)
    data = handle_missing_airPressure_values(data)
    data = handle_missing_windDir_values(data)

    data = data.set_index(keys = ['year','month','week','day','hour']).drop(columns = 'index')
    
    return data

In [ ]:
# DO NOT MODIFY OR COPY THIS CELL!!
weather_data_complete = handle_missing_values_weather(weather_data)

In [ ]:
print(f"Before: \n---------\n{weather_data.isna().sum()}")
print(f"\nAfter: \n---------\n{weather_data_complete.isna().sum()}")

print(weather_data_complete.shape)
print(weather_data.shape)

In [ ]:
# tests, DO NOT MODIFY OR COPY THIS CELL!!
# check if missing values are no longer present
assert not weather_data_complete.isna().any().any()
assert weather_data_complete.shape[0] == weather_data.shape[0]
assert weather_data_complete.shape[1] == weather_data.shape[1]

## Task 3: Handling Outliers (10 Points)

Additionally to the missing values, the dataset also seems to have some strange values, that are probably outliers. 
When confronted with the data, the meterologist gave you a bit more information:


1. Sometimes the temperture readings seem to be off, without any good reason.

2. In the timespan from early October 2015 until mid March 2016, the wind sensor was defective: it might have displayed wrong values for winds from SE direction.

3. In the `daily_weather_data`, the precipitation column contains some non-numeric values.

4. The precipitation sensor usually produces wrong values when hail is involved.






### 3.1 Temperature outliers
First we want ot take a closer look at the temperature values. Check if we can identify some obvious outliers and come up with a strategy to handle/fix them.

In order to do so you will have to:
- Plot the temperature curve over time and a histogram of temperature values to identify possible outliers
- Plot a zoomed in version of individual outliers to get a better understanding what's happening
- Devise a strategy to get rid of outliers

#### Investigation
Implement the function below to create a plot of the temperature values (`temp`) over time. Additionally create a histogram with reasonable bins to identify the outliers:

In [ ]:
def plot_temp_analysis(df: pd.DataFrame) -> None:
    """
    Create two plots:
    1) Temperature values over time for the whole dataframe
    2) A histogram for temperature values. 
       Choose appropriate bins enabling you to identify outliers!

    Parameters
    --------
    df: data frame containint the temperature values (temp) with potential outlier
    
    """

    # YOUR CODE HERE

    df = df.sort_values(by = 'date')
    
    fig, ax = plt.subplots()

    ax.plot(df['date'], df['temp'])
    
    plt.show()


    fig, ax = plt.subplots()

    plt.hist(df['temp'], range = (60,90))

    plt.show()
    


In [ ]:
# DO NOT MODIFY OR COPY THIS CELL!!
plot_temp_analysis(weather_data_complete)

In [ ]:
# hidden test, DO NOT MODIFY OR COPY THIS CELL!!

In the next cell, select a random outlier (e.g. the first) and plot the temperature curve around the outlier.

In [ ]:
# YOUR CODE HERE
from pandas.tseries.offsets import DateOffset

outlier = weather_data_complete[weather_data_complete['temp'] > 60]['date'].to_list()[5]

ts = pd.Timestamp(outlier)

start_date = ts - DateOffset(days = 10)
end_date = ts + DateOffset(days = 10)


outliers = weather_data_complete[(weather_data_complete['date'] >= start_date) & (weather_data_complete['date'] <= end_date)][['date','temp']].sort_values(by = 'date')

fig, ax = plt.subplots()

ax.plot(outliers['date'], outliers['temp'])

plt.show()


In [ ]:
# hidden test, DO NOT MODIFY OR COPY THIS CELL!!

#### Remove temperature outliers
Implement the below function using the strategy you defined above to get rid of the temperature outliers

In [ ]:
def handle_temp_outliers(noisy_data) -> pd.DataFrame:
    """
    Parameters
    --------
    noisy_data: data frame that contains temperature outliers ('temp' column)

    Returns
    --------
    cleaned_data: data frame with temperature outliers removed/handled
    """
    cleaned_data = noisy_data.copy()

    # YOUR CODE HERE
    Q3 = noisy_data['temp'].quantile(0.75)
    Q1 = noisy_data['temp'].quantile(0.25)

    IQR = Q3 - Q1

    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    cleaned_data['temp'] = cleaned_data['temp'].clip(lower = lower, upper = upper)


    return cleaned_data

In [ ]:
# DO NOT MODIFY OR COPY THIS CELL!!
weather_data_cleaned = handle_temp_outliers(weather_data_complete)
plot_temp_analysis(weather_data_cleaned)
print(weather_data_cleaned['temp'].describe())

In [ ]:
# test, DO NOT MODIFY OR COPY THIS CELL!!

# check if new missing values were introduced
assert not weather_data_cleaned.isna().any().any()
# check if outliers were simply dropped
assert weather_data_cleaned.shape == weather_data_complete.shape

In [ ]:
# hidden test, DO NOT MODIFY OR COPY THIS CELL!!

In [ ]:
# hidden test, DO NOT MODIFY OR COPY THIS CELL!!

### 3.2 Wind speed outliers

The second outlier problem was, that in the timespan from early October 2015 until mid March 2016, the wind sensor was defective: it might have displayed wrong values for winds from SE direction.

Double check if this is true, and if it is, fix the values appropriately.

#### Data inspection

Implement the function below to visualize the problematic wind sensor data.
Complete the function below to check the assumtions for wrong values of windBeauf.
- Plot the wind speed data over the questionable time duration (early October 2015 until mid March 2016)
- Plot a histogram of the windspeed values to identify outliers
- Check if the assumtion regarding the wind direction is true (errors only from SE direction)

Bonus:
- Find out when exactly the sensor started to produce wrong values and when the sensor was fixed!

In [ ]:
def analyze_wind_range(df:pd.DataFrame) -> typing.Tuple[int, typing.List[str]]:
    """
    Plot analysis plots for the wind data. Print outputs about relevant data 
    and return the relevant values as indicated.

    Parameters
    --------
    noisy_data: data frame that contains outliers
    
    Returns
    --------
    wind_dir_outliers: list of wind direction string code, for which outliers where found
    num_outliers: number of found outliers

    """

    num_outliers = 0  # return a value that reflects how many outliers you identified
    wind_dir_outliers = [] # a list of the string codes for wind directions where you identified outliers from


    # YOUR CODE HERE
    start_date = pd.Timestamp(year = 2015, month = 10, day = 1)
    end_date = pd.Timestamp(year = 2016, month = 3, day = 15)

    time_in_question = df[(df['date'] >= start_date) & (df['date'] <= end_date)].sort_values(by = 'date')

    fig, ax = plt.subplots()

    ax.plot(time_in_question['date'], time_in_question['windBeauf'])
    
    plt.show()



    # Find Outliers with IQR

    Q3 = time_in_question['windBeauf'].quantile(0.75)
    Q1 = time_in_question['windBeauf'].quantile(0.25)

    IQR = Q3 - Q1

    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    outliers = time_in_question[(time_in_question['windBeauf'] <= lower) | (time_in_question['windBeauf'] >= upper)]

    num_outliers = (outliers.count()[0])

    wind_dir_outliers = outliers['windDir'].to_list()

    return wind_dir_outliers, num_outliers


In [ ]:
# DO NOT MODIFY OR COPY THIS CELL!!
wind_dir_outliers, num_outliers = analyze_wind_range(weather_data_complete)

In [ ]:
assert 0 < num_outliers < 500, "there should be more than zero but less than 500 outliers!"
assert len(wind_dir_outliers) > 0, "there should be at least one wind direction!"
assert isinstance(wind_dir_outliers[0], str), "the wind direction codes are strings with two characters!"
assert len(wind_dir_outliers[0]) == 2, "the wind direction codes are strings with two characters!"

# hidden test, DO NOT MODIFY OR COPY THIS CELL!!

#### Fix wind sensor outliers
Fix the values appropriately. Implement a function that compensates for the problem you found.

In [ ]:
def fix_windBeauf_values(df:pd.DataFrame) -> pd.DataFrame:
    """
    Parameters
    --------
    df: data frame that contains potential faulty wind values

    Returns
    -------
    df_ret: data frame with fixed wind values
    """

    df_ret = df.copy()

    Q3 = df_ret['windBeauf'].quantile(0.75)
    Q1 = df_ret['windBeauf'].quantile(0.25)

    IQR = Q3 - Q1

    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    df_ret['windBeauf'] = df_ret['windBeauf'].clip(lower = lower, upper = upper)

    return df_ret

In [ ]:
# test, DO NOT MODIFY OR COPY THIS CELL!!
weather_data_fix_wind = fix_windBeauf_values(weather_data_complete)
wind_dir_outliers_fixed, num_outliers_fixed = analyze_wind_range(weather_data_fix_wind)

In [ ]:
# test, DO NOT MODIFY OR COPY THIS CELL!!
assert num_outliers_fixed == 0, "now no outliers should be found"
assert len(wind_dir_outliers_fixed) == 0, "now no outliers should be found, so no directions!"
assert weather_data_fix_wind.shape == weather_data_complete.shape

# hidden test, DO NOT MODIFY OR COPY THIS CELL!!

### 3.3 Daily weather data: precipitation

When loading the data, we separated the precipitation data into the `daily_weather_data` dataframe.
This dataframe also has an issues:

- The `precip` column contains some non-numeric values 

#### Data inspection
Check the occurance of the non-numeric values in the precipitation data. You can check the file `data/weather/description.txt`, which might have additional clues what is going on.

Implement the function below and return a list of the non-numeric values that occur in the `precip` column of `daily_weather_data`. Make sure to only return every unique value once! 

In [ ]:
def get_non_numeric_precip_values(df:pd.DataFrame) -> typing.Set[str]:
    """
    Parameters
    --------
    df: data frame that contains non-numeric values in precip column

    Returns
    -------
    non_numeric_values: list of unique non-numeric values. 
    Do not return duplicate values in the list!
    """
    non_numeric_values = set()

    vals = set(df['precip'].unique())

    for val in vals:
        try:
            float(val)
        except:
            non_numeric_values.add(val)
    

    return non_numeric_values

In [ ]:
# DO NOT MODIFY OR COPY THIS CELL!!
non_numeric_values = get_non_numeric_precip_values(daily_weather_data)
print(f"\nnon-numeric values values: {non_numeric_values}")

In [ ]:
# tests, DO NOT MODIFY OR COPY THIS CELL!!
assert isinstance(non_numeric_values, set) , "make sure to return a set, so no duplicate values can be returned!!"
assert len(non_numeric_values) > 0, "there should be some non-numeric values in daily_weather_data!"
assert isinstance(list(non_numeric_values)[0], str), "only return the non-numeric values as strings!"

#### Fix non-numeric values
Replace non-numeric values with some appropriate numerical values and convert the column to a more suitable data type.
To get an idea, what appropriate values might be, check the file `data/weather/description.txt` and the other numeric values in the `precip` column.

In [ ]:
def fix_precip_values(df:pd.DataFrame) -> pd.DataFrame:
    """
    Parameters
    --------
    df: data frame that contains non-numeric values in precip column

    Returns
    -------
    ret_df: data frame with fixed precip values
    """
    ret_df = df.copy()

    # YOUR CODE HERE
    ret_df['precip'] = ret_df['precip'].replace('traces', 0.01)

    ret_df['precip'] = ret_df['precip'].apply(pd.to_numeric)

    return ret_df

In [ ]:
# DO NOT MODIFY OR COPY THIS CELL!!
daily_weather_data_fixed_precip = fix_precip_values(daily_weather_data)

In [ ]:
# test, DO NOT MODIFY OR COPY THIS CELL!!
assert pd.api.types.is_float_dtype(daily_weather_data_fixed_precip['precip'].dtype), "precip should now be a float column!!"
assert daily_weather_data_fixed_precip.shape == daily_weather_data.shape, "do not remove or add rows!"

### Combining the fixes

In [ ]:
# DO NOT MODIFY OR COPY THIS CELL!!
def fix_values_daily(data):
    """ 
    Parameters
    --------
    data: data frame containing missing values 
    
    Returns
    --------
    complete_data: data frame not containing any missing values
    """
    complete_data = data.copy()
    complete_data = fix_precip_values(complete_data)
    
    return complete_data

   
def handle_outliers(data):
    """ 
    Parameters
    --------
    data: data frame containing outlier values 
    
    Returns
    --------
    complete_data: data frame not containing any outlier values
    """
    complete_data = data.copy()
    complete_data = handle_temp_outliers(complete_data)
    complete_data = fix_windBeauf_values(complete_data)
    
    return complete_data

In [ ]:
# DO NOT MODIFY OR COPY THIS CELL!!
daily_weather_data_finished = fix_values_daily(daily_weather_data)
weather_data_finished = handle_outliers(weather_data_complete)

## Task 4: Aggregate values (5 Points)

Aggregate the observations on a weekly basis. Return a data frame with a hierarchical index (levels `year` and `week`) and the following weekly aggregations as columns:

- `temp_weeklyMin`: minimum of `temp`
- `temp_weeklyMax`: max of `temp`
- `temp_weeklyMean`: mean of `temp`
- `temp_weeklyMedian`: median of `temp`

- `hum_weeklyMin`: min of `hum`
- `hum_weeklyMax`: max of `hum`
- `hum_weeklyMean`: mean of `hum`

- `wind_weeklyMean`: mean of `windBeauf`
- `wind_weeklyMax`: max of `windBeauf`
- `wind_weeklyMin`: min of `windBeauf`

Additionally merge the precipitation values from the `daily_weather_data` dataframe also into the newly created dataframe and aggregate them into the folling columns:
- `precip_weeklyMin`: min of `precip`
- `precip_weeklyMax`: max of `precip`
- `precip_weeklyMean`: mean of `precip`


**Note:** Attentive data scientists might have noticed a problem with isocalendars when aggregating on `Year` and `Week`. You can ignore this for the purpose of this lecture. In real-world settings you might consider addressing this issue, depending on your task and data

In [ ]:
def aggregate_weekly(hourly_data, data_daily):
    """ 
    Parameters
    --------
    hourly_data: hourly weather data frame, containing temp, hum, and wind values.
    data_daily: daily weather data frame with precip values
    
    Returns
    --------
    weekly_stats: data frame that contains statistics aggregated on a weekly basis
    """
    weekly_weather_data = pd.DataFrame()

    # YOUR CODE HERE
    weekly_weather_data['temp_weeklyMin'] = hourly_data.groupby(by = ['year','week'])['temp'].min()
    weekly_weather_data['temp_weeklyMax'] = hourly_data.groupby(by = ['year','week'])['temp'].max()
    weekly_weather_data['temp_weeklyMean'] = hourly_data.groupby(by = ['year','week'])['temp'].mean()
    weekly_weather_data['temp_weeklyMedian'] = hourly_data.groupby(by = ['year','week'])['temp'].median()
    
    weekly_weather_data['hum_weeklyMin'] = hourly_data.groupby(by = ['year','week'])['hum'].min()
    weekly_weather_data['hum_weeklyMax'] = hourly_data.groupby(by = ['year','week'])['hum'].max()
    weekly_weather_data['hum_weeklyMean'] = hourly_data.groupby(by = ['year','week'])['hum'].mean()

    weekly_weather_data['wind_weeklyMin'] = hourly_data.groupby(by = ['year','week'])['windBeauf'].min()
    weekly_weather_data['wind_weeklyMax'] = hourly_data.groupby(by = ['year','week'])['windBeauf'].max()
    weekly_weather_data['wind_weeklyMean'] = hourly_data.groupby(by = ['year','week'])['windBeauf'].mean()

    weekly_weather_data['precip_weeklyMin'] = data_daily.groupby(by = ['year','week'])['precip'].min()
    weekly_weather_data['precip_weeklyMax'] = data_daily.groupby(by = ['year','week'])['precip'].max()
    weekly_weather_data['precip_weeklyMean'] = data_daily.groupby(by = ['year','week'])['precip'].mean()

    return weekly_weather_data

In [ ]:
# DO NOT MODIFY OR COPY THIS CELL!!
weekly_weather_data = aggregate_weekly(weather_data_finished, daily_weather_data_finished)
display(weekly_weather_data)

In [ ]:
# tests, DO NOT MODIFY OR COPY THIS CELL!!
assert len(weekly_weather_data.columns) >= 13, "according to the instructions, the dataframe should have >= 13 columns"
assert len(weekly_weather_data.index.levels) == 2, "according to the instructions, the dataframe should have a multi-index with 2 levels"

In [ ]:
# hidden tests, DO NOT MODIFY OR COPY THIS CELL!!

## Task 5: Merge influenza and weather datasets (5 Points)

Merge the `data_weather_weekly` and `data_influenza` datasets.
Both dataframes should now be on a weekly index. 
Beware that both datasets contain rows that do not appear in the other dataset.

In [ ]:
# tests, DO NOT MODIFY OR COPY THIS CELL!!
# Neither of the tables contain missing data

print(f"influenza data, missing data (should be 0): \n{data_influenza.isna().sum()}")
print(f"weather data, missing data (should be 0): \n{weekly_weather_data.isna().sum()}")

assert not data_influenza.isna().any().any(), "we should have eliminated all missing values!!"
assert not weekly_weather_data.isna().any().any(), "we should have eliminated all missing values!!"

In [ ]:
# use this cell for experimentation / analysis for merging data

# YOUR CODE HERE
weekly_weather_data
# data_influenza

In [ ]:
def merge_data(weather_df, influenza_df):
    """ 
    Parameters
    --------
    weather_df: weekly weather data frame
    influenza_df: influenza data frame
    
    Returns
    --------
    merged_data: merged data frame that contains both weekly weather observations and prevalence of influence infections
    """
    # YOUR CODE HERE
    merged_data = weather_df.merge(influenza_df, how = 'inner', left_index = True, right_index = True)

    return merged_data

In [ ]:
# DO NOT MODIFY OR COPY THIS CELL!!
data_merged = merge_data(weekly_weather_data, data_influenza)
data_merged.head()

In [ ]:
# tests, DO NOT MODIFY OR COPY THIS CELL!!
print(data_merged.shape)
assert data_merged.shape[0] > 300, "there should be more than 300 rows in the merged dataset"
assert data_merged.shape[1] >= 14, "1 column for infections, 13 feature columns from weather data"
